In [12]:
import nipype
import nipype.interfaces.utility as niu
import nipype.interfaces.io as nio
from nipype.interfaces import fsl, freesurfer, ants
from nipype.pipeline import engine as pe

import os

In [13]:
# utility function to checkout registration results
def make_report(fixed_image, moving_image, 
                fn='report.svg',
                compress_report=False):
    from nipype.interfaces.base import File
    _fixed_image = fixed_image
    _moving_image = moving_image
    _fixed_image_mask = None
    _fixed_image_label = "fixed"
    _moving_image_label = "moving"
    _contour = None
    # _out_report = File('report.svg', usedefault=True, desc='filename for the visual report')#'report.svg'
    #output_dir=os.getcwd()
    import os
    _out_report = os.path.abspath(os.path.join(os.getcwd(), fn))
    
    from uuid import uuid4
    from niworkflows.viz.utils import plot_registration, compose_view, cuts_from_bbox, robust_set_limits, extract_svg
    from nilearn.masking import apply_mask, unmask
    from nilearn.image import threshold_img, load_img
    
    def plot_registration_with_overlay(
        anat_nii,
        div_id,
        overlay_nii,
        plot_params=None,
        order=("z", "x", "y"),
        cuts=None,
        estimate_brightness=False,
        label=None,
        contour=None,
        compress="auto",
        dismiss_affine=False,
    ):
        """
        Plots the foreground and background views
        Default order is: axial, coronal, sagittal
        """
        from svgutils.transform import fromstring
        from nilearn.plotting import plot_anat
        from nilearn import image as nlimage
        import matplotlib.pyplot as plt

        plot_params = {} if plot_params is None else plot_params

        # Use default MNI cuts if none defined
        if cuts is None:
            raise NotImplementedError  # TODO

        out_files = []
        if estimate_brightness:
            plot_params = robust_set_limits(anat_nii.get_fdata().reshape(-1), plot_params)
            plot_params_overlay = robust_set_limits(overlay_nii.get_fdata().reshape(-1), {})

        # FreeSurfer ribbon.mgz
        ribbon = contour is not None and np.array_equal(
            np.unique(contour.get_fdata()), [0, 2, 3, 41, 42]
        )

        if ribbon:
            contour_data = contour.get_fdata() % 39
            white = nlimage.new_img_like(contour, contour_data == 2)
            pial = nlimage.new_img_like(contour, contour_data >= 2)

        if dismiss_affine:
            canonical_r = rotation2canonical(anat_nii)
            anat_nii = rotate_affine(anat_nii, rot=canonical_r)
            if ribbon:
                white = rotate_affine(white, rot=canonical_r)
                pial = rotate_affine(pial, rot=canonical_r)
            if contour:
                contour = rotate_affine(contour, rot=canonical_r)

        # Plot each cut axis
        for i, mode in enumerate(list(order)):
            plot_params["display_mode"] = mode
            plot_params["cut_coords"] = cuts[mode]
            if i == 0:
                plot_params["title"] = label
            else:
                plot_params["title"] = None

            # Generate nilearn figure
            display = plot_anat(anat_nii, **plot_params)
            if ribbon:
                kwargs = {"levels": [0.5], "linewidths": 0.5}
                display.add_contours(white, colors="b", **kwargs)
                display.add_contours(pial, colors="r", **kwargs)
            elif contour is not None:
                display.add_contours(contour, colors="r", levels=[0.5], linewidths=0.5)
            display.add_overlay(overlay_nii, cmap=plt.cm.gray, vmin=plot_params_overlay['vmin'], vmax=plot_params_overlay['vmax'])

            svg = extract_svg(display, compress=compress)
            display.close()

            # Find and replace the figure_1 id.
            svg = svg.replace("figure_1", "%s-%s-%s" % (div_id, mode, uuid4()), 1)
            out_files.append(fromstring(svg))

        return out_files

    
    fixed_image_nii = load_img(_fixed_image)  # template
    moving_image_nii = load_img(_moving_image)  # source
    contour_nii = load_img(_contour) if _contour is not None else None

    if _fixed_image_mask:
        fixed_image_nii = unmask(apply_mask(fixed_image_nii,
                                            _fixed_image_mask),
                                 _fixed_image_mask)
        # since the moving image is already in the fixed image space we
        # should apply the same mask
        moving_image_nii = unmask(apply_mask(moving_image_nii,
                                             _fixed_image_mask),
                                  _fixed_image_mask)
        mask_nii = load_img(_fixed_image_mask)
    else:
        mask_nii = threshold_img(fixed_image_nii, 1e-3)

    n_cuts = 7
    if not _fixed_image_mask and contour_nii:
        cuts = cuts_from_bbox(contour_nii, cuts=n_cuts)
    else:
        cuts = cuts_from_bbox(mask_nii, cuts=n_cuts)

    # Call composer
    compose_view(
        plot_registration(fixed_image_nii, 'fixed-image',
                          estimate_brightness=True,
                          cuts=cuts,
                          label=_fixed_image_label,
                          contour=contour_nii,
                          compress=compress_report),
        plot_registration_with_overlay(fixed_image_nii, 'moving-image', moving_image_nii,
                          estimate_brightness=True,
                          cuts=cuts,
                          label=_moving_image_label,
                          contour=contour_nii,
                          compress=compress_report),
        # plot_registration(moving_image_nii, 'moving-image',
        #                   estimate_brightness=True,
        #                   cuts=cuts,
        #                   label=_moving_image_label,
        #                   contour=contour_nii,
        #                   compress=compress_report),
        out_file=_out_report
    )
    return _out_report



In [5]:
subject_ids = [str(x).zfill(2) for x in range(1,20)]
subject_ids

['01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19']

### Registration: BBR

In [6]:
from niworkflows.interfaces.freesurfer import (
    PatchedBBRegisterRPT as BBRegisterRPT,
    PatchedMRICoregRPT as MRICoregRPT,
    PatchedLTAConvert as LTAConvert
)
from niworkflows.interfaces.nitransforms import ConcatenateXFMs

In [14]:
## pipeline
project_folder = '/home/stevenm/Projects/bias'
workflow = pe.Workflow(name='coregister_flash_to_mp2rage')
workflow.base_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# identity
identity = pe.Node(niu.IdentityInterface(fields=['subject_id']), name='identity')
identity.iterables = [('subject_id', subject_ids),]

# selector
templates = {'t1w': os.path.join(project_folder, 'derivatives', 'ds-01', 'fmriprep', 'sub-{subject_id}',  'anat',  'sub-{subject_id}_desc-preproc_T1w.nii.gz'),
             'flash_echoes': os.path.join(project_folder, 'sourcedata', 'sourcedata', 'ds-01', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_echo-*_FLASH.nii.gz'),
             'brain_mask': os.path.join(project_folder, 'derivatives', 'ds-01', 'fmriprep', 'sub-{subject_id}',  'anat',  'sub-{subject_id}_desc-brain_mask.nii.gz'),
             'fsnative2t1w_xfm': os.path.join(project_folder, 'derivatives', 'ds-01', 'fmriprep', 'sub-{subject_id}',  'anat',  'sub-{subject_id}_from-fsnative_to-T1w_mode-image_xfm.txt'),
             }

selector = pe.Node(nio.SelectFiles(templates), name='selector')

workflow.connect(identity, 'subject_id', selector, 'subject_id')


# 1. FLASH mean image: merge in t, then meants
from nipype.interfaces.fsl import Merge

merger = pe.Node(fsl.Merge(), 'flash_merger')
merger.inputs.dimension = 't'
merger.inputs.output_type = 'NIFTI_GZ'

workflow.connect(selector, 'flash_echoes', merger, 'in_files')

# 2. mean image
mean_ts = pe.Node(fsl.ImageMaths(), 'flash_meaner')
mean_ts.inputs.op_string = '-Tmean'

workflow.connect(merger, 'merged_file', mean_ts, 'in_file')


# 3. bet mean flash
better = pe.Node(fsl.BET(), 'flash_better')
better.inputs.padding = True
better.inputs.frac = 0.2

workflow.connect(mean_ts, 'out_file', better, 'in_file')

# 4. BET with padding messes up the affine for no reason, so fix
def fix_affine(img1, img2):
    import nibabel as nib
    import os
    output_dir = os.getcwd()
    new_img = nib.Nifti1Image(nib.load(img1).get_fdata(), header=nib.load(img1).header, affine=nib.load(img2).affine)
    fn = os.path.join(output_dir, 'fixed_affine.nii.gz')
    
    new_img.to_filename(fn)
    return fn
affine_fixer = pe.Node(niu.Function(input_names=['img1', 'img2'], output_names=['out_file'], function=fix_affine), 'affine_fixer')
workflow.connect(better, 'out_file', affine_fixer, 'img1')
workflow.connect(mean_ts, 'out_file', affine_fixer, 'img2')


# 5. BBregister
def generate_sub_string(sub):
    return f'sub-{sub}'

string_generator = pe.Node(niu.Function(input_names=['sub'], output_names=['sub_str'], function=generate_sub_string), 'string_gen')

workflow.connect(identity, 'subject_id', string_generator, 'sub')

bbreg = pe.Node(freesurfer.BBRegister(init='fsl', contrast_type='t2'), 'flash_bbreg')
bbreg.inputs.subjects_dir = os.path.join(project_folder, 'derivatives/ds-01/freesurfer')
bbreg.inputs.out_fsl_file = True
bbreg.inputs.out_lta_file = True
bbreg.inputs.registered_file = True

workflow.connect(affine_fixer, 'out_file', bbreg, 'source_file')
workflow.connect(string_generator, 'sub_str', bbreg, 'subject_id')

## 6. Registration to itk (ants format). Code from fmriprep.
transforms = pe.Node(niu.Merge(2), run_without_submitting=True, name='transforms')
lta_ras2ras = pe.MapNode(LTAConvert(out_lta=True), iterfield=['in_lta'],
                         name='lta_ras2ras', mem_gb=2)
# In cases where Merge(2) only has `in1` or `in2` defined
# output list will just contain a single element
select_transform = pe.Node(
    niu.Select(index=0),
    run_without_submitting=True,
    name='select_transform'
)
merge_ltas = pe.Node(niu.Merge(2), name='merge_ltas', run_without_submitting=True)
concat_xfm = pe.Node(ConcatenateXFMs(inverse=True), name='concat_xfm')

workflow.connect([
    (bbreg, transforms, [('out_lta_file', 'in1')]),
    (selector, merge_ltas, [('fsnative2t1w_xfm', 'in2')]),
    # Wire up the co-registration alternatives
    (transforms, lta_ras2ras, [('out', 'in_lta')]),
    (lta_ras2ras, select_transform, [('out_lta', 'inlist')]),
    (select_transform, merge_ltas, [('out', 'in1')]),
    (merge_ltas, concat_xfm, [('out', 'in_xfms')]),
])


# 7. Plot coregistration results
# first, mask the skull
brain_masker = pe.Node(fsl.ImageMaths(), 'brain_masker')
workflow.connect(selector, 'brain_mask', brain_masker, 'mask_file')
workflow.connect(selector, 't1w', brain_masker, 'in_file')

reporter = pe.Node(niu.Function(input_names=['fixed_image', 'moving_image'], output_names=['out_report'], function=make_report), name='reporter')
workflow.connect(brain_masker, 'out_file', reporter, 'fixed_image')
workflow.connect(bbreg, 'registered_file', reporter, 'moving_image')


# 8. datasink
# sink: (1) registered mean flash; (2) xfm forward, (3) xfm inverse
ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = os.path.join(project_folder, 'derivatives', 'ds-01', 'coregister_flash_to_t1w')

ds.inputs.regexp_substitutions = [('itk_flash_to_t1/_subject_id_([0-9]{2})/out_fwd.tfm', 'sub-\\1/sub-\\1_from-FLASH_to-T1w_mode-image_xfm.txt'),
                                  ('itk_t1_to_flash/_subject_id_([0-9]{2})/out_inv.tfm', 'sub-\\1/sub-\\1_from-T1w_to-FLASH_mode-image_xfm.txt'),
                                  ('registered_mean_flash/_subject_id_([0-9]{2})/fixed_affine_bbreg.nii.gz', 'sub-\\1/sub-\\1_space-T1w_desc-mean-FLASH.nii.gz'),
                                  ('registration_report/_subject_id_([0-9]{2})/report.svg', 'sub-\\1/sub-\\1_space-T1w_desc-mean-FLASH.svg')]

workflow.connect(bbreg, 'registered_file', ds, 'registered_mean_flash')
workflow.connect(reporter, 'out_report', ds, 'registration_report')
workflow.connect(concat_xfm, 'out_inv', ds, 'itk_t1_to_flash')
workflow.connect(concat_xfm, 'out_xfm', ds, 'itk_flash_to_t1')

SyntaxError: invalid syntax (<ipython-input-14-3068f3b13d4b>, line 133)

In [11]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs':20, 'memory_gb': 250})

220303-14:56:31,565 nipype.workflow INFO:
	 Workflow coregister_flash_to_mp2rage settings: ['check', 'execution', 'logging', 'monitoring']
220303-14:56:32,428 nipype.workflow INFO:
	 Running in parallel.
220303-14:56:32,459 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 38 jobs ready. Free memory (GB): 250.00/250.00, Free processors: 20/20.
220303-14:56:32,842 nipype.workflow INFO:
	 [Job 0] Cached (coregister_flash_to_mp2rage.string_gen).
220303-14:56:32,872 nipype.workflow INFO:
	 [Node] Outdated cache found for "coregister_flash_to_mp2rage.selector".
220303-14:56:33,206 nipype.workflow INFO:
	 [Job 15] Cached (coregister_flash_to_mp2rage.string_gen).
220303-14:56:33,212 nipype.workflow INFO:
	 [Node] Outdated cache found for "coregister_flash_to_mp2rage.selector".
220303-14:56:33,214 nipype.workflow INFO:
	 [Job 30] Cached (coregister_flash_to_mp2rage.string_gen).
220303-14:56:33,216 nipype.workflow INFO:
	 [Node] Outdated cache found for "coregister_flash_to_mp2rage.selec

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/nitransforms/base.py:207: UserWarning: Reference space not set
  warnings.warn("Reference space not set")


220303-15:11:34,980 nipype.workflow INFO:
	 [Node] Finished "coregister_flash_to_mp2rage.concat_xfm".


/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


220303-15:11:35,943 nipype.workflow INFO:
	 [Job 13] Completed (coregister_flash_to_mp2rage.concat_xfm).
220303-15:11:35,947 nipype.workflow INFO:
	 [MultiProc] Running 19 tasks, and 1 jobs ready. Free memory (GB): 246.20/250.00, Free processors: 1/20.
                     Currently running:
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.flash_bbreg
                       * coregister_flash_to_mp2rage.flash_bbreg
                       * coregister_flash_to_mp2rage.flash_bbreg
                       * coregister_flash_to_mp2rage.flash_bbreg
                       * coregister_flash_to_mp2rage.flash_bbreg
                       * coregister_flash_to_mp2rage.flash_bbreg
                       * coregister_flash_to_mp2rage.flash_bbreg
                       * coregister_flash_to_mp2rage.flash_bbreg
                       * coregister_flash_to_mp2rage.flash_bbre

...

220303-15:18:36,148 nipype.workflow INFO:
	 [Node] Finished "coregister_flash_to_mp2rage.concat_xfm".
220303-15:18:36,897 nipype.workflow INFO:
	 [Job 148] Completed (coregister_flash_to_mp2rage.concat_xfm).
220303-15:18:36,899 nipype.workflow INFO:
	 [Job 283] Completed (coregister_flash_to_mp2rage.concat_xfm).
220303-15:18:36,902 nipype.workflow INFO:
	 [MultiProc] Running 17 tasks, and 0 jobs ready. Free memory (GB): 246.60/250.00, Free processors: 3/20.
                     Currently running:
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
  

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


220303-15:19:10,972 nipype.workflow INFO:
	 [MultiProc] Running 16 tasks, and 1 jobs ready. Free memory (GB): 246.80/250.00, Free processors: 4/20.
                     Currently running:
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
      

/home/stevenm/.conda/envs/python3/lib/python3.7/site-packages/nitransforms/base.py:207: UserWarning: Reference space not set
  warnings.warn("Reference space not set")


220303-15:19:11,632 nipype.workflow INFO:
	 [Node] Finished "coregister_flash_to_mp2rage.concat_xfm".
220303-15:19:12,966 nipype.workflow INFO:
	 [Job 58] Completed (coregister_flash_to_mp2rage.concat_xfm).
220303-15:19:12,972 nipype.workflow INFO:
	 [MultiProc] Running 16 tasks, and 0 jobs ready. Free memory (GB): 246.80/250.00, Free processors: 4/20.
                     Currently running:
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp2rage.reporter
                       * coregister_flash_to_mp

In [ ]:
# ## pipeline
# project_folder = '/home/stevenm/Projects/bias'
# workflow = pe.Workflow(name='coregister_flash_to_mp2rage')
# workflow.base_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
# workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# # identity
# identity = pe.Node(niu.IdentityInterface(fields=['subject_id']), name='identity')
# identity.iterables = [('subject_id', subject_ids),]

# # selector
# templates = {'t1w': os.path.join(project_folder, 'derivatives', 'ds-01', 'fmriprep', 'sub-{subject_id}',  'anat',  'sub-{subject_id}_desc-preproc_T1w.nii.gz'),
#              'flash_echoes': os.path.join(project_folder, 'sourcedata', 'sourcedata', 'ds-01', 'sub-{subject_id}', 'anat', 'sub-{subject_id}_echo-*_FLASH.nii.gz'),
#              'brain_mask': os.path.join(project_folder, 'derivatives', 'ds-01', 'fmriprep', 'sub-{subject_id}',  'anat',  'sub-{subject_id}_desc-brain_mask.nii.gz'),
#              }

# selector = pe.Node(nio.SelectFiles(templates), name='selector')

# workflow.connect(identity, 'subject_id', selector, 'subject_id')


# # 1. FLASH mean image: merge in t, then meants
# from nipype.interfaces.fsl import Merge

# merger = pe.Node(fsl.Merge(), 'flash_merger')
# merger.inputs.dimension = 't'
# merger.inputs.output_type = 'NIFTI_GZ'

# workflow.connect(selector, 'flash_echoes', merger, 'in_files')

# # 2. mean image
# mean_ts = pe.Node(fsl.ImageMaths(), 'flash_meaner')
# mean_ts.inputs.op_string = '-Tmean'

# workflow.connect(merger, 'merged_file', mean_ts, 'in_file')


# # 3. bet mean flash
# better = pe.Node(fsl.BET(), 'flash_better')
# better.inputs.padding = True
# better.inputs.frac = 0.2

# workflow.connect(mean_ts, 'out_file', better, 'in_file')

# # 4. BET with padding messes up the affine for no reason, so fix
# def fix_affine(img1, img2):
#     import nibabel as nib
#     import os
#     output_dir = os.getcwd()
#     new_img = nib.Nifti1Image(nib.load(img1).get_fdata(), header=nib.load(img1).header, affine=nib.load(img2).affine)
#     fn = os.path.join(output_dir, 'fixed_affine.nii.gz')
    
#     new_img.to_filename(fn)
#     return fn
# affine_fixer = pe.Node(niu.Function(input_names=['img1', 'img2'], output_names=['out_file'], function=fix_affine), 'affine_fixer')
# workflow.connect(better, 'out_file', affine_fixer, 'img1')
# workflow.connect(mean_ts, 'out_file', affine_fixer, 'img2')


# # # 5. To RAS - don't do this here
# # from nipype.interfaces.image import Reorient
# # reorient = pe.Node(Reorient(orientation='RAS'), 'reorienter')
# # workflow.connect(affine_fixer, 'out_file', reorient, 'in_file')


# # 6. BBregister
# def generate_sub_string(sub):
#     return f'sub-{sub}'

# string_generator = pe.Node(niu.Function(input_names=['sub'], output_names=['sub_str'], function=generate_sub_string), 'string_gen')

# workflow.connect(identity, 'subject_id', string_generator, 'sub')

# bbreg = pe.Node(freesurfer.BBRegister(init='fsl', contrast_type='t2'), 'flash_bbreg')
# bbreg.inputs.subjects_dir = os.path.join(project_folder, 'derivatives/ds-01/freesurfer')
# bbreg.inputs.out_fsl_file = True
# bbreg.inputs.out_lta_file = True
# bbreg.inputs.registered_file = True

# workflow.connect(affine_fixer, 'out_file', bbreg, 'source_file')
# workflow.connect(string_generator, 'sub_str', bbreg, 'subject_id')

# # 7. invert XFM?
# convert = pe.Node(fsl.ConvertXFM(invert_xfm=True), 'xfm_inverter')
# workflow.connect(bbreg, 'out_fsl_file', convert, 'in_file')

# # 


# # Plot coregistration results
# # first, mask the skull
# brain_masker = pe.Node(fsl.ImageMaths(), 'brain_masker')
# workflow.connect(selector, 'brain_mask', brain_masker, 'mask_file')
# workflow.connect(selector, 't1w', brain_masker, 'in_file')

# reporter = pe.Node(niu.Function(input_names=['fixed_image', 'moving_image'], output_names=['out_report'], function=make_report), name='reporter')
# workflow.connect(brain_masker, 'out_file', reporter, 'fixed_image')
# workflow.connect(bbreg, 'registered_file', reporter, 'moving_image')


# # 8. datasink
# # sink: (1) registered mean flash; (2) xfm forward, (3) xfm inverse
# ds = pe.Node(nio.DataSink(), name='datasink')
# ds.inputs.base_directory = os.path.join(project_folder, 'derivatives', 'ds-01', 'coregister_flash_to_t1w')

# workflow.connect(bbreg, 'out_fsl_file', ds, 'forward_xfm')
# workflow.connect(convert, 'out_file', ds, 'inverse_xfm')
# workflow.connect(bbreg, 'out_lta_file', ds, 'forward_lta')
# workflow.connect(bbreg, 'registered_file', ds, 'registered_mean_flash')
# workflow.connect(reporter, 'out_report', ds, 'registration_report')